In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("002 RL.Ecommerce").getOrCreate()

In [2]:
datos=spark.read.csv("rl/Ecommerce_Customers.csv",header=True
                     ,inferSchema=True)

In [3]:
datos.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [4]:
datos.head(2)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)]

In [5]:
print(datos.head(1)[0][0])

mstephenson@fernandez.com


In [6]:
for elemento in datos.head(1)[0]:
    print(elemento)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [7]:
datos.count()

500

In [29]:
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [30]:
datos.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [31]:
constructor=VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol="features")

In [32]:
salida=constructor.transform(datos)

In [33]:
salida.first()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [13]:
datosFinales=salida.select("features","Yearly Amount Spent")
datosFinales.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [14]:
from pyspark.ml.regression import LinearRegression

In [15]:
datosEntrena,datosEvalua=datosFinales.randomSplit([0.7,0.3])

In [16]:
datosEntrena.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                349|
|   mean| 498.38441816490075|
| stddev|  79.76610333213642|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [17]:
datosEvalua.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                151|
|   mean| 501.46263039566236|
| stddev|   78.4827833586748|
|    min|  275.9184206503857|
|    max|  744.2218671047146|
+-------+-------------------+



In [18]:
lr=LinearRegression(labelCol="Yearly Amount Spent")

In [19]:
modelo=lr.fit(datosEntrena)

In [20]:
modelo.coefficients

DenseVector([25.7428, 39.0806, 0.3935, 61.4619])

In [21]:
modelo.summary.r2

0.985264393963961

In [22]:
modelo.summary.rootMeanSquaredError

9.668949372421093

In [23]:
evalua=modelo.evaluate(datosEvalua)

In [24]:
evalua.r2

0.9819429779328407

In [25]:
evalua.predictions.show()

+--------------------+-------------------+------------------+
|            features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[29.5324289670579...|  408.6403510726275|397.63054857053476|
|[31.0472221394875...|  392.4973991890214|387.58049722386227|
|[31.3091926408918...|  432.7207178399336|429.73137162781836|
|[31.3895854806643...|  410.0696110599829| 409.0265397879243|
|[31.4252268808548...|  530.7667186547619| 535.0737590578913|
|[31.5171218025062...|  275.9184206503857|  279.933211029067|
|[31.5257524169682...|  443.9656268098819|449.14146105261193|
|[31.5702008293202...|  545.9454921414049| 563.9598795554691|
|[31.5741380228732...|  544.4092721605869| 558.8957714040998|
|[31.6253601348306...|  376.3369007569242|381.52905180030666|
|[31.7207699002873...|   538.774933478023| 545.4126599436765|
|[31.8124825597242...|  392.8103449837972| 396.0174710891549|
|[31.8186165667690...| 446.41867337013565|448.30272398939906|
|[31.820

In [26]:
evalua.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.009802502092725|
|  4.916901965159127|
|  2.989346212115265|
| 1.0430712720585689|
| -4.307040403129349|
| -4.014790378681312|
| -5.175834242730048|
| -18.01438741406423|
| -14.48649924351298|
|-5.1921510433824665|
|   -6.6377264656536|
| -3.207126105357702|
|-1.8840506192634052|
|  8.034763561796524|
|-17.173345691822362|
| 12.037997205933038|
|-13.690470806232497|
|0.48658576989663516|
| -8.788200585876439|
|  18.15521572222042|
+-------------------+
only showing top 20 rows



In [27]:
predice=modelo.transform(salida)

In [28]:
predice.select("Email","Time on Website","prediction").show()

+--------------------+------------------+------------------+
|               Email|   Time on Website|        prediction|
+--------------------+------------------+------------------+
|mstephenson@ferna...| 39.57766801952616| 594.7294596867241|
|   hduke@hotmail.com|37.268958868297744|380.02124801397645|
|    pallen@yahoo.com|37.110597442120856| 504.7893975804809|
|riverarebecca@gma...| 36.72128267790313| 571.0149322733816|
|mstephens@davidso...| 37.53665330059473| 591.7010227937742|
|alvareznancy@luca...| 34.47687762925054|   638.74622470276|
|katherine20@yahoo...| 36.68377615286961| 516.4977140819572|
|  awatkins@yahoo.com| 37.37335885854755| 558.3477800354833|
|vchurch@walter-ma...|37.534497341555735| 559.6270761856222|
|    bonnie69@lin.biz| 37.14516822352819| 440.8898557015607|
|andrew06@peterson...| 37.22580613162114| 509.1765422804833|
|ryanwerner@freema...| 37.08792607098381|  513.288201581616|
|   knelson@gmail.com| 37.42021557502538|397.63054857053476|
|wrightpeter@yahoo...|36